In [1]:
import numpy as np

a = np.asarray([[1,2,3],[4,5,6]]);
b = np.asarray([[10]]);

res = a*b;

print(res);

import sys;
print(sys.path)


[[10 20 30]
 [40 50 60]]
['', '/Users/norbertmauger/anaconda/envs/py35/lib/python35.zip', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/plat-darwin', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/lib-dynload', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/Sphinx-1.3.5-py3.5.egg', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/setuptools-20.3-py3.5.egg', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/aeosa', '/Users/norbertmauger/anaconda/envs/py35/lib/python3.5/site-packages/IPython/extensions', '/Users/norbertmauger/.ipython']


In [11]:
import numpy as np;

import theano.tensor as T;
from theano import function;
from theano import pp;

x = T.scalar('x');
y = T.scalar('y');

z = x + y;
z2 = x * y;
print(pp(z));
f = function([x,y],z)
f2 = function([x,y],[z,z2])




(x + y)


In [35]:
print(f(2,3))
print(f2(2,3))
np.allclose(f(2.34,5.67), 8.01)

np.allclose([3,3.9,5], [3,4,5.1],atol=0.1)


5.0
[array(5.0), array(6.0)]


True

In [39]:
x = T.dmatrix('x');
y = T.dmatrix('y');

z = x + y;
f = function([x,y],z)

f([[2,3],[4,7]], [[30,10],[50,30]])

array([[ 32.,  13.],
       [ 54.,  37.]])

In [45]:
import theano
a = theano.tensor.vector()
b = theano.tensor.vector()
out = a**2 + b**2 + 2*a*b
f = theano.function([a,b],out)
f([3,1,34],[4,1,321])

array([  4.90000000e+01,   4.00000000e+00,   1.26025000e+05])

In [58]:
import theano
import theano.tensor as T
x = T.matrix('x')
s = 1 / (1 + T.exp(-x))

sigmoid = function([x], s)
sigmoid([[0,1,-1000],[2,3,1000]])

array([[ 0.5       ,  0.73105858,  0.        ],
       [ 0.88079708,  0.95257413,  1.        ]])

In [65]:
# possible de créer plusieurs variable de type matrice en même temps via dmatrices
# Possible ensuite de faire faire plusieurs calculs simultanément à une fonction
a, b = T.dmatrices('a','b')
diff = a - b
absdiff = abs(diff)
sqrdiff = diff**2

dif_func = function([a,b], [diff, absdiff, sqrdiff])

dif_func([[2,3]],[[2,5]])

[array([[ 0., -2.]]), array([[ 0.,  2.]]), array([[ 0.,  4.]])]